### Class decorators: customizing class creation

Recall that

```python
@foo
def func(...):
    ...
```

really means

```python
def func(...):
    ...
func = foo(func)
```

In [ ]:
def decorator_factory(a, b):
    def decorator(function):
        print(f'Decorating {function} with ({a}, {b}): {id(decorator)}')
        def wrapper(*args, **kwargs):
            print('Calling decorated function', a, b)
            result = function(*args, **kwargs)
            ...
            return result
        return wrapper
    print(f'Creating decorator with args({a}, {b}): {id(decorator)}')
    return decorator

In [ ]:
deco = decorator_factory(1,2)

In [ ]:
@deco
def my_cool_function(c, d, e):
    print('cool: ', c, d, e)

In [ ]:
my_cool_function

In [ ]:
my_cool_function(10, 20, e=256)

In [ ]:
@decorator_factory(1,2)
def my_cool_function(c, d, e):
    print('cool: ', c, d, e)

In [ ]:
my_cool_function(5,6,7)

```python
@locking(the_vault)
def transfer_money(a, b, amount):
    pass
```

## So... let's decorate classes, as well...

We can do the same with classes:

```python
@foo
class Bar:
    ...
```

means

```python
class Bar:
    ...
Bar = foo(Bar)
```

### Class decorators use case: create a registry of classes

In [ ]:
class Registry:
    
    def __init__(self):
        self._registry = {}

    def register(self, cls):
        """This is to be used as a class decorator...."""
        self._registry[cls.__name__] = cls
        return cls

    def __getitem__(self, name):
        return self._registry[name]
    
r = Registry()

In [ ]:
# "Application" code
@r.register
class Registered1:
    pass

#Registered1 = r.register(Registered1)

@r.register
class Registered2:
    pass

In [ ]:
r._registry

In [ ]:
r['Registered1']

In [ ]:
r['Registered2']

Real-world examples: [Flask-RESTPlus][rest+], [Flask-SMOREST][smorest]

[rest+]: https://flask-restplus.readthedocs.io/en/stable/quickstart.html#a-minimal-api
[smorest]: https://flask-smorest.readthedocs.io/en/latest/quickstart.html

### Class decorators use case: collect 'important' fields (ORM, schema library, etc.)

We would like to write something like the following and have the class 'know' what the important fields are:

```python
@declarative
class MyClass:
    def __repr__(self):
        return f"<MyClass {' '.join(repr(i) for i in self._important)} >"
    
    a = ImportantField(1)
    b = ImportantField(2)
    c = ImportantField(3, 'cfield')
```

In [ ]:
class ImportantField:
    def __init__(self, value, name=None):
        self.value = value
        self.name = name
        
    def __repr__(self):
        return f'<Important {self.name}: {self.value}>'
        
        
def declarative(cls):
    print('Running declarative() decorator')
    _important = []
    for name in dir(cls):
        value = getattr(cls, name)
        if isinstance(value, ImportantField):
            if value.name is None:
                value.name = name
            _important.append(value)
    cls._important = _important
    return cls
    
@declarative
class MyClass:
    def __repr__(self):
        return f"<MyClass {' '.join(repr(i) for i in self._important)} >"
    
    a = ImportantField(1)
    b = ImportantField(2)
    c = ImportantField(3, 'cfield')    

In [ ]:
obj = MyClass()

In [ ]:
obj

In [ ]:
MyClass._important

### Class decorators are _not_ inherited:

In [ ]:
class OtherClass(MyClass):
    d = ImportantField(5)

In [ ]:
OtherClass()

In [ ]:
@declarative
class OtherClass(MyClass):
    d = ImportantField(5)

In [ ]:
OtherClass()

## Use case/example: introduce a level of inheritance

Example from Barin: 
    
```python
def cmap(collection):
    '''decorator that marks a class as providing behavior for a collection'''
    def decorator(cls):
        mapped_cls = type(
            cls.__name__, (cls, collection), {})
        collection.m.registry.register_override(collection, mapped_cls)
        return mapped_cls
    return decorator


@cmap(class_collection)
class MyClass(library.MySuperClass):
    a = SomeThing()
    b = SomeThing()
```

Another real-world example: [DataClasses][dataclasses]

[dataclasses]: https://docs.python.org/3/library/dataclasses.html

Open [Class Decorators Lab](./class-decorators-lab.ipynb)